In [103]:
import torch
import numpy as np
from scipy.io import loadmat
from pathlib import Path
import torchsummary
import matplotlib.pyplot as plt
import matplotlib
# from torch.optim.lr_scheduler import StepLR
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from scipy.interpolate import griddata
import pandas as pd

def setup_plot(figsize=(8,10)):
    plt.rc('font', family='serif', serif = "cmr10", size=20)
    plt.rcParams['mathtext.fontset']='cm'
    matplotlib.rcParams['axes.unicode_minus'] = False
    plt.rcParams['axes.formatter.use_mathtext'] = True
    plt.figure(figsize=figsize)

def save_and_close(file='plot.png'):
    plt.tight_layout()
    plt.savefig(file)
    plt.close()

def numerical_data(filename, dtype=torch.float32, device='cpu'):
    data = np.genfromtxt(filename, delimiter=',', skip_header=1)
    y_vector = data[:, 0]
    Q_vector = data[:, 1]
    k_vector = data[:, 2]
    return torch.tensor(np.column_stack((y_vector, Q_vector, k_vector)),
                        dtype=dtype).to(device)

def makePlots(model):
    output_Qkp = model(model.dataAll_y)
    output_Q = output_Qkp[:,0:1].detach().numpy()
    output_k = output_Qkp[:,1:2].detach().numpy()
    output_P = output_Qkp[:,2:3].detach().numpy()
    plot_y = model.dataAll_y.detach().numpy()

    data_Q = model.dataAll_Q.detach().numpy()
    data_k = model.dataAll_k.detach().numpy()

    setup_plot()
    # Create a 2x2 grid of subplots
    _, axs = plt.subplots(2, 2, figsize=(8, 8))
    # Plot data on each subplot
    axs[0, 0].plot(plot_y, output_Q, label='PINN')
    axs[0, 0].plot(plot_y, data_Q, label='data')
    axs[0, 0].set_ylabel('$Q$')
    axs[0, 0].set_xlabel('$y$')
    axs[0, 0].legend()

    axs[0, 1].plot(plot_y, output_k, label='PINN')
    axs[0, 1].plot(plot_y, data_k, label='data')
    axs[0, 1].set_ylabel('$k$')
    axs[0, 1].set_xlabel('$y$')
    axs[0, 1].legend()

    axs[1, 0].plot(plot_y, output_P)
    axs[1, 0].set_ylabel('$P$')
    axs[1, 0].set_xlabel('$y$')

    loss_info_history = np.array(model.loss_history)
    loss_history = loss_info_history[:,0]
    loss_data_history = loss_info_history[:,1]
    loss_eqn_history = loss_info_history[:,2]
    loss_bd_history = loss_info_history[:,3]

    axs[1, 1].plot(loss_data_history, label='data loss', linewidth=2, alpha=0.7)
    axs[1, 1].plot(loss_eqn_history, label='eqn loss', linewidth=2, alpha=0.7)
    axs[1, 1].plot(loss_bd_history, label='bd loss', linewidth=2, alpha=0.7)
    axs[1, 1].plot(loss_history, label='loss', linewidth=2, alpha=0.7)
    axs[1, 1].set_xlabel('Iteration')
    axs[1, 1].set_yscale('log')
    legend = axs[1, 1].legend(fontsize=12)
    for line in legend.legend_handles: line.set_linewidth(2)

    save_and_close()

'''if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("GPU not available, using the CPU instead.")'''
#device = torch.device("mps")
device = torch.device("cpu")

In [105]:

class NeuralNet(nn.Module):
    def __init__(self, layers, output_activation=None):
        super(NeuralNet, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))
        self.output_activation = output_activation

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = torch.tanh(layer(x))
        x = self.layers[-1](x)
        if self.output_activation:
            x = self.output_activation(x)
        return x

class PINN_structure(nn.Module):
    def __init__(self, layers_Qk, layers_P, device, data, weights):
       
        super(PINN_structure, self).__init__()
        self.device = device
        
        self.dataAll_y = data[:, 0:1].to(device)
        self.dataAll_Q = data[:, 1:2].to(device)
        self.dataAll_k = data[:, 2:3].to(device)
        
        self.net_Qk = NeuralNet(layers_Qk).to(device)
        self.net_P = NeuralNet(layers_P).to(device)
        #self.net_Qk = NeuralNet(layers_Qk, output_activation= torch.relu ).to(device)
        #self.net_P = NeuralNet(layers_P, output_activation = torch.relu ).to(device)
        #self.net_Qk = NeuralNet(layers_Qk, output_activation= torch.softmax ).to(device)
        #self.net_P = NeuralNet(layers_P, output_activation = torch.softmax ).to(device)

        self.ld, self.le, self.lb = weights[0], weights[1], weights[2] 
        
        self.loss_history = []
        self.term_history = []
        
        self.data_type = self.dataAll_y.dtype
        self.loss0 = torch.tensor([1]).to(device)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        self.iteration = 0
        
# ------------------------------------------------------------------------------------------------------------------
# Forward Method
# ------------------------------------------------------------------------------------------------------------------

    def forward(self, input_y):
        output_Qk = self.net_Qk(input_y)
        output_Q = output_Qk[:, 0:1]
        output_k = output_Qk[:, 1:2]
        output_P = self.net_P(input_y)
        return torch.cat([output_Q, output_k, output_P], dim=1)

# ------------------------------------------------------------------------------------------------------------------
# Loss computing, equation loss, boundary loss, and the total loss
# ------------------------------------------------------------------------------------------------------------------

    def gov_eqn(self, input_y):
        input_y.requires_grad_(True)
        
        outputs = self.forward(input_y)
        output_Q, output_k, output_P = outputs[:, 0:1], outputs[:, 1:2], outputs[:, 2:3]

        dP_dy = torch.autograd.grad(output_P, input_y,
                        torch.ones_like(output_P), create_graph=True)[0]
        dkdpdy_dy = torch.autograd.grad(output_k*dP_dy, input_y,
                        torch.ones_like(output_P), create_graph=True)[0] # d/dy(k*dp/dy)

        return output_Q + dkdpdy_dy

    def dirichlet_boundary_condition(self, boundary_y, boundary_value):
        outputs = self.forward(boundary_y)
        output_P = outputs[:, 2:3]
        return torch.abs(output_P - boundary_value)
    
    def compute_loss(self,):

        y_collocation = torch.tensor((np.random.uniform(
            low=-1, high=1, size=1234)), dtype=self.data_type).to(device)[np.newaxis,:].T
        #y_collocation = self.dataAll_y # simple way. collocation points are just the data points

        mseloss = nn.MSELoss()
        output_QkP = self.forward(self.dataAll_y)
        output_Q = output_QkP[:, 0:1]
        output_k = output_QkP[:, 1:2]
        #output_P = output_QkP[:, 2:3]
        data_error = mseloss(output_Q, self.dataAll_Q) + mseloss(output_k, self.dataAll_k)
        
        f_equation = self.gov_eqn(y_collocation)
        equation_error = torch.mean(f_equation**2, dim=0)

        #f_boundary_y0 = self.dirichlet_boundary_condition(torch.zeros(1), 1)
        #boundary_error = torch.mean(f_boundary_y0**2, dim=0)
        boundary_error = torch.zeros(1)

        data_weight = torch.tensor([1.0], dtype=self.data_type).to(device)
        eqn_weight = torch.tensor([1.0], dtype=self.data_type).to(device)
        boundary_weight = torch.tensor([1.0], dtype=self.data_type).to(device)

        loss_data = torch.sum(data_error * data_weight)
        loss_equation = torch.sum(equation_error * eqn_weight)
        loss_boundary = torch.sum(boundary_error * boundary_weight)

        loss = (self.ld * loss_data + self.le * loss_equation + self.lb * loss_boundary) / self.loss0

        return loss, loss_data, loss_equation, loss_boundary, data_error, equation_error, boundary_error
    
# ------------------------------------------------------------------------------------------------------------------
# Model training
# ------------------------------------------------------------------------------------------------------------------
    
    def train_model(self, nIter, print_every_iter = 100):
        # self.iteration = 0
        for it in range(nIter):
            self.optimizer.zero_grad()  # Zero the gradients
            [loss, loss_data, loss_equation, loss_boundary, data_error, equation_error,
                boundary_error] = self.compute_loss()
            loss_value = loss  # Assuming self.loss0 is defined elsewhere
            loss_value.backward()  # Compute gradients
            self.optimizer.step()  # Update parameters

            # Log and print loss
            loss_info = [loss_value.item(), loss_data.item(), loss_equation.item(), loss_boundary.item()]
            self.loss_history.append(loss_info + [data_error.item(), equation_error.item(), boundary_error.item()])
            if self.iteration % print_every_iter == 0:
                print(f'iteration:{self.iteration:06d}, l_value:{loss_info[0]:.6f}, ld:{loss_info[1]:.6f}, '
                      f'le:{loss_info[2]:.6f}, lb:{loss_info[3]:.6f}')
            self.iteration += 1

layers_Qk = [1, 20, 20, 20, 2]
layers_P = [1, 20, 20, 20, 1]

# Initialize the weight for the loss computation
weights = torch.tensor([1, 0.2, 0.2]).to(device)

data = numerical_data(filename='Forward_Data.txt', device=device)
model = PINN_structure(layers_Qk, layers_P, device, data, weights,)
model.to(device)


PINN_structure(
  (net_Qk): NeuralNet(
    (layers): ModuleList(
      (0): Linear(in_features=1, out_features=20, bias=True)
      (1-2): 2 x Linear(in_features=20, out_features=20, bias=True)
      (3): Linear(in_features=20, out_features=2, bias=True)
    )
  )
  (net_P): NeuralNet(
    (layers): ModuleList(
      (0): Linear(in_features=1, out_features=20, bias=True)
      (1-2): 2 x Linear(in_features=20, out_features=20, bias=True)
      (3): Linear(in_features=20, out_features=1, bias=True)
    )
  )
)

In [106]:
# Let's train the model
rounds = 200
niter = 20
model.requires_grad_(True)

# Wrap the outer loop with tqdm for a progress bar
for each_round in tqdm(range(rounds), desc='Training Rounds'):
    model.train_model(niter)
    makePlots(model)

torch.save({
    'iteration': model.iteration,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': model.optimizer.state_dict(),
    'loss': model.loss_history,
    'val':model.term_history,
    # any other items you wish to save
}, 'checkpoint.pth')

Training Rounds:   0%|          | 0/200 [00:00<?, ?it/s]

iteration:000000, l_value:2.120702, ld:2.120598, le:0.000519, lb:0.000000


Training Rounds:   2%|▎         | 5/200 [00:02<01:12,  2.68it/s]

iteration:000100, l_value:0.014366, ld:0.010262, le:0.020520, lb:0.000000


Training Rounds:   5%|▌         | 10/200 [00:03<01:04,  2.95it/s]

...

<Figure size 800x1000 with 0 Axes>

<Figure size 800x1000 with 0 Axes>

<Figure size 800x1000 with 0 Axes>

<Figure size 800x1000 with 0 Axes>

<Figure size 800x1000 with 0 Axes>

In [102]:
output_Qkp = model(model.dataAll_y)
output_Q = output_Qkp[:,0:1].detach().numpy()
output_k = output_Qkp[:,1:2].detach().numpy()
output_P = output_Qkp[:,2:3].detach().numpy()
plot_y = model.dataAll_y.detach().numpy()

data_Q = model.dataAll_Q.detach().numpy()
data_k = model.dataAll_k.detach().numpy()

setup_plot()
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
# Plot data on each subplot
axs[0, 0].plot(plot_y, output_Q, label='PINN')
axs[0, 0].plot(plot_y, data_Q, label='data')
axs[0, 0].set_ylabel('$Q$')
axs[0, 0].set_xlabel('$y$')
axs[0, 0].legend()

axs[0, 1].plot(plot_y, output_k, label='PINN')
axs[0, 1].plot(plot_y, data_k, label='data')
axs[0, 1].set_ylabel('$k$')
axs[0, 1].set_xlabel('$y$')
axs[0, 1].legend()

axs[1, 0].plot(plot_y, output_P)
axs[1, 0].set_ylabel('$P$')
axs[1, 0].set_xlabel('$y$')

loss_info_history = np.array(model.loss_history)
loss_history = loss_info_history[:,0]
loss_data_history = loss_info_history[:,1]
loss_eqn_history = loss_info_history[:,2]
loss_bd_history = loss_info_history[:,3]

axs[1, 1].plot(loss_data_history, label='data loss', linewidth=2, alpha=0.7)
axs[1, 1].plot(loss_eqn_history, label='eqn loss', linewidth=2, alpha=0.7)
axs[1, 1].plot(loss_bd_history, label='bd loss', linewidth=2, alpha=0.7)
axs[1, 1].plot(loss_history, label='loss', linewidth=2, alpha=0.7)
axs[1, 1].set_xlabel('Iteration')
axs[1, 1].set_yscale('log')
legend = axs[1, 1].legend(fontsize=12)
for line in legend.legend_handles: line.set_linewidth(2)

save_and_close()

<Figure size 800x1000 with 0 Axes>